In [ ]:
import ccxt
import os
import logging
import time
from dotenv import load_dotenv
from tabulate import tabulate
import pandas as pd

load_dotenv()


class Configuration:
    @staticmethod
    def fetch_api_credentials():
        try:
            # Fetch API credentials from environment variables
            API_KEY = os.getenv('API_KEY')
            SECRET_KEY = os.getenv('SECRET_KEY')
            PASSPHRASE = os.getenv('PASSPHRASE')

            return API_KEY, SECRET_KEY, PASSPHRASE

        except Exception as e:
            raise RuntimeError(f"Error fetching API credentials: {e}")


class MarketMaker:
    symbol = 'TON/USDT'  # Set your trading symbol
    historical_data_path = 'ton_MMdata.csv'

    def __init__(self):
        self.exchange = self.init_exchange()
        self.logger = self.setup_logger()
        self.accumulated_profit = 0.0

        # Fetch order book to calculate initial slippage
        bids, asks = self.fetch_order_book()
        slippage = self.calculate_slippage(bids, asks)

        # Initial data for the historical data DataFrame
        initial_data = {
            'Timestamp': [time.time()],
            'Slippage': [slippage],
            'Available Capital': [self.print_balance()],
            'Proposed Entry Bid Price': [0.0],
            'Proposed Entry Ask Price': [0.0],
            'Potential Profit (Buy)': [0.0],
            'Potential Profit (Sell)': [0.0],
            'Accumulated Profit': [self.accumulated_profit]
        }

        # Create the initial historical data DataFrame
        self.historical_data = pd.DataFrame(initial_data)

        # Write initial data to CSV
        self.write_to_csv()

    def init_exchange(self):
        try:
            API_KEY, SECRET_KEY, PASSPHRASE = Configuration.fetch_api_credentials()

            # Initialize the KuCoin Futures exchange instance
            exchange = ccxt.kucoin({
                'apiKey': API_KEY,
                'secret': SECRET_KEY,
                'password': PASSPHRASE,
                'enableRateLimit': True  # Adjust as needed
            })

            return exchange

        except ccxt.ExchangeError as e:
            self.handle_exception(f"Exchange initialization error: {e}")
        except Exception as e:
            self.handle_exception(f"An unexpected error occurred during exchange initialization: {e}")

    def setup_logger(self):
        logger = logging.getLogger('market_maker')
        logger.setLevel(logging.INFO)

        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        ch.setFormatter(formatter)

        logger.addHandler(ch)

        return logger

    def handle_exception(self, message):
        self.logger.error(f"An error occurred: {message}")

    def fetch_order_book(self, symbol='TON/USDT'):
        try:
            order_book = self.exchange.fetch_order_book(symbol)
            bids = order_book['bids']
            asks = order_book['asks']

            return bids, asks

        except ccxt.NetworkError as e:
            self.handle_exception(f"Network error: {e}")
        except ccxt.ExchangeError as e:
            self.handle_exception(f"Exchange error: {e}")
        except Exception as e:
            self.handle_exception(f"An unexpected error occurred: {e}")

    def calculate_slippage(self, bids, asks):
        best_bid = bids[0][0]
        best_ask = asks[0][0]
        slippage = best_ask - best_bid
        return slippage

    def print_balance(self):
        available_capital = 'N/A'
        try:
            balance = self.exchange.fetch_balance()
            available_capital = balance['total'].get('USDT', 'N/A')
            self.logger.info(f"Available Capital: {available_capital} USDT")  # Printing available capital

        except ccxt.NetworkError as e:
            self.handle_exception(f"Network error: {e}")
        except ccxt.ExchangeError as e:
            self.handle_exception(f"Exchange error: {e}")
        except Exception as e:
            self.handle_exception(f"An error occurred: {e}")

        return available_capital

    def load_historical_data(self):
        if os.path.exists(self.historical_data_path):
            return pd.read_csv(self.historical_data_path)
        else:
            return pd.DataFrame(columns=['Timestamp', 'Slippage', 'Available Capital',
                                         'Proposed Entry Bid Price', 'Proposed Entry Ask Price',
                                         'Potential Profit (Buy)', 'Potential Profit (Sell)',
                                         'Accumulated Profit'])

    def write_to_csv(self):
        # Append the current data to the CSV file
        self.historical_data.to_csv(self.historical_data_path, index=False, mode='a', header=not os.path.exists(self.historical_data_path))

    def aggregate_prices(self, prices, num_levels):
        return [[sum(price[0] for price in prices[:i + 1]) / (i + 1), prices[i][1]]
                for i in range(min(num_levels, len(prices)))]

    def calculate_best_bids_asks(self, bids, asks, num_levels=5):
        aggregated_bids = self.aggregate_prices(bids, num_levels)
        aggregated_asks = self.aggregate_prices(asks, num_levels)

        return aggregated_bids, aggregated_asks

    def display_best_bids_asks(self, best_bids, best_asks):
        df_bids = pd.DataFrame(best_bids, columns=['Price', 'Volume'])
        df_asks = pd.DataFrame(best_asks, columns=['Price', 'Volume'])

        # Concatenate DataFrames horizontally
        df_combined = pd.concat([df_bids, df_asks], axis=1, keys=['Best Bids', 'Best Asks'])

        # Display the combined DataFrame using tabulate
        self.logger.info(tabulate(df_combined, headers='keys', tablefmt='pretty', showindex=False))  # Logging best bids and asks

    def calculate_trading_amount(self):
        try:
            # Fetch balance
            balance = self.exchange.fetch_balance()

            # Print available capital
            available_capital = balance['total'].get('USDT', 'N/A')
            self.logger.info(f"Available Capital: {available_capital} USDT")

            # Calculate trading amount as a percentage of available capital (0.14%)
            trading_percentage = 0.0034
            trading_amount = available_capital * trading_percentage

            # Print the trading amount
            self.logger.info(f"Trading Amount (0.34%): {trading_amount} USDT")

            return trading_amount
        except ccxt.NetworkError as e:
            self.logger.error(f"Network error: {e}")
        except ccxt.ExchangeError as e:
            self.logger.error(f"Exchange error: {e}")
        except Exception as e:
            self.logger.error(f"An error occurred: {e}")

        return None  # Return None if an error occurs

    def calculate_potential_profit(self, entry_price, slippage, side):
        # Updated trading fees
        maker_fee = 0.001  # 0.1%
        taker_fee = 0.001  # 0.1%

        if side == 'buy':
            # Calculate profit percentage
            profit_percentage = (entry_price + slippage) / entry_price - 1
        elif side == 'sell':
            # Calculate profit percentage
            profit_percentage = 1 - (entry_price - slippage) / entry_price
        else:
            self.logger.warning("Invalid side. Cannot calculate profit.")
            return None

        trading_amount = self.calculate_trading_amount()

        # Deduct trading fees from the trading amount only once
        if side == 'buy':
            trading_amount -= trading_amount * maker_fee
        elif side == 'sell':
            trading_amount -= trading_amount * taker_fee

        potential_profit = trading_amount * profit_percentage

        self.logger.info(f"Potential Profit ({side}): {potential_profit} USDT")
        return potential_profit

    def create_limit_order(self, side, entry_price, trading_amount):
        try:
            # Create limit order
            main_order = self.exchange.create_order(
                self.symbol,
                type='limit',
                side=side,
                amount=trading_amount,
                price=entry_price,
                params={
                    'postOnly': False,
                    'timeInForce': 'GTC',
                    # You can add more parameters as needed
                }
            )

            # Fetch order details for confirmation
            order_details = self.exchange.fetch_order(main_order['id'])

            # Log order details
            self.logger.info(f"Main Order Created: {main_order}")
            self.logger.info(f"Order Details: {order_details}")

        except ccxt.NetworkError as e:
            self.logger.error(f"Network error: {e}")
        except ccxt.ExchangeError as e:
            self.logger.error(f"Exchange error: {e}")
        except Exception as e:
            self.logger.error(f"An error occurred: {e}")

    def run_market_maker(self):
        last_display_time = 0

        while True:
            try:
                bids, asks = self.fetch_order_book()
                slippage = self.calculate_slippage(bids, asks)
                self.logger.info(f"Slippage: {slippage}")  # Logging slippage

                available_capital = self.print_balance()

                best_bids, best_asks = self.calculate_best_bids_asks(bids, asks)
                if time.time() - last_display_time >= 60:
                    self.display_best_bids_asks(best_bids, best_asks)  # Logging best bids and asks
                    last_display_time = time.time()

                    # Calculate proposed entry prices
                    entry_bid_price = best_bids[0][0] + slippage
                    entry_ask_price = best_asks[0][0] - slippage

                    self.logger.info(f"Proposed Entry Bid Price: {entry_bid_price}")  # Logging proposed entry bid price
                    self.logger.info(f"Proposed Entry Ask Price: {entry_ask_price}")  # Logging proposed entry ask price

                    # Calculate and print potential profit
                    potential_profit_buy = self.calculate_potential_profit(entry_bid_price, slippage, 'buy')
                    potential_profit_sell = self.calculate_potential_profit(entry_ask_price, slippage, 'sell')

                    self.accumulated_profit += potential_profit_buy + potential_profit_sell
                    self.logger.info(f"Potential Profit (Buy): {potential_profit_buy} USDT")
                    self.logger.info(f"Potential Profit (Sell): {potential_profit_sell}")
                    self.logger.info(f"Accumulated Profit: {self.accumulated_profit} USDT")

                    # Update the historical data DataFrame
                    new_data = {
                        'Timestamp': [time.time()],
                        'Slippage': [slippage],
                        'Available Capital': [available_capital],
                        'Proposed Entry Bid Price': [entry_bid_price],
                        'Proposed Entry Ask Price': [entry_ask_price],
                        'Potential Profit (Buy)': [potential_profit_buy],
                        'Potential Profit (Sell)': [potential_profit_sell],
                        'Accumulated Profit': [self.accumulated_profit]
                    }

                    # Convert the new data to a DataFrame
                    new_data_df = pd.DataFrame(new_data)

                    # Concatenate the new data to the existing historical data DataFrame
                    self.historical_data = pd.concat([self.historical_data, new_data_df], ignore_index=True)

                    # Write updated data to CSV
                    self.write_to_csv()

                    # Place limit orders
                    self.create_limit_order('buy', entry_bid_price, self.calculate_trading_amount())
                    self.create_limit_order('sell', entry_ask_price, self.calculate_trading_amount())

            except Exception as e:
                self.handle_exception(f"An unexpected error occurred: {e}")

            time.sleep(1)  # Adjust the sleep time as needed


if __name__ == "__main__":
    market_maker_instance = MarketMaker()
    market_maker_instance.run_market_maker()
